# Static maps
***

**_Autor:_** Chus Casado Rodríguez<br>
**_Fecha:_** 02-04-2024<br>

**Introduction:**<br>
This notebook creates the files that make up the ResOpsES dataset.

**To do:**<br>
* [ ] Fix `point_polygon_statistics` to work with masks rasters instead of polygons, so that we can compute the attributes of reservoirs and lakes.
* [ ] Would it be possible to use the new `lisflood-utilities.catchstats.catchment_statistics` function instead of `mask_statistics`?
* [ ] In the [LISFLOOD parameters](#LISFLOOD-parameters), it could be interesting to extract the reservoir parameters for each of the reservoirs represented in EFAS.
* [ ] There's 1 day lag between EFAS5 and EMO1. EMO1 starts January 2nd 1990, whereas EFAS5 January 1st.

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
# import rioxarray
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.features import shapes
from typing import Union, List, Dict, Optional, Tuple
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime

from funciones import mask_statistics, read_static_map, plot_attributes#, dict2da

## Configuration

In [ ]:
# path to the dataset ResOpsES
path_datasets = Path('Z:/nahaUsers/casadje/datasets')
path_ResOpsES = path_datasets / 'reservoirs' / 'ResOpsES'

In [ ]:
# path to EFASv5
path_EFAS = Path('E:/casadje/Documents/EFASv5/iberia')

In [ ]:
# crs = 4326

In [ ]:
# beginning of the study period: start of the EFAS5 long run
start = datetime(1990, 1, 1)

## Reservoirs

In [ ]:
# load reservoir points
reservoirs = gpd.read_file(path_ResOpsES / 'GIS' / 'reservoirs_ResOpsES.shp')
reservoirs.set_index('SNCZI', inplace=True)
reservoirs.index = reservoirs.index.astype(int)
reservoirs.CAP_MCM = reservoirs.CAP_MCM.astype(float)
reservoirs.CATCH_SKM = reservoirs.CATCH_SKM.astype(float)

print('{0} reservoirs and {1} attributes'.format(*reservoirs.shape))

# plot reservoir locations
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', edgecolor='face', facecolor='lightgray'),
               alpha=.5,
               zorder=0)
reservoirs.plot(markersize=reservoirs.CAP_MCM**.5, cmap='viridis', c=reservoirs.CATCH_SKM, alpha=.5, ax=ax)
ax.set_aspect('equal')
ax.axis('off');

## Catchments

In [ ]:
path = path_ResOpsES / 'ancillary' / 'cutmaps'
mask_map = 'upArea_01min.nc'

masks = {}
directories = [dir for dir in path.iterdir() if dir.is_dir()]
for directory in tqdm(directories, desc='loading masks'):
    try:
        # reservoir ID
        ID = int(directory.stem)
        
        # load upstream area map
        mask = xr.open_dataset(directory / mask_map)['Band1']
        mask.name = str(ID)
        
        # create and save a mask out of the previous map
        mask = xr.where(mask.notnull(), 1, mask)
        masks[ID] = mask
    except:
        print(directory)
        continue

In [ ]:
# export masks as NetCDF with ID as filename to be used with `catchstats`
path_masks = path_ResOpsES / 'ancillary' / 'cutmaps' / 'masks'
path_masks.mkdir(parents=True, exist_ok=True)
for ID, mask in masks.items():
    mask.to_netcdf(path_masks / f'{ID:04}.nc')

## Static maps

In this section I will compute catchment statistics of the LISFLOOD static maps that will be in the end exported as _attributes_EFAS_static_maps.csv_. As ancillary maps, I will load first the pixel area and upstream area maps, that will be needed in the subsequent calculations.

In [ ]:
# load the pixel area map
pixarea = xr.open_mfdataset(f'{path_EFAS}/maps/pixarea*.nc')['Band1'].compute()

# load the upstream area map
upArea = xr.open_mfdataset(f'{path_EFAS}/maps/upArea*.nc')['Band1'].compute()

### Geomorphology

Here I will compute catchment statistics for geomorphological attributes

In [ ]:
func = {'elv': ['mean', 'std', 'min', 'max'],
        'gradient': ['mean', 'std'],
        'upArea': ['max'],
        # 'pixarea': ['sum']
       }

# cargar mapas
geomorphology = xr.Dataset({var: xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')['Band1'].compute() for var in func})

# calcular estadísticos
attr_geomorphology = mask_statistics(geomorphology, masks, func, weight=pixarea).to_pandas()

# plot attributes
plot_attributes(attr_geomorphology, reservoirs.geometry.x, reservoirs.geometry.y, ncols=4)

### Land use

In [ ]:
variables = ['fracforest', 'fracirrigated', 'fracother', 'fracrice', 'fracwater', 'fracsealed']
variables.sort()

# load maps
land_use = xr.Dataset({var: xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')['Band1'].compute() for var in variables})
land_use = land_use.rename({var: var[4:] for var in list(land_use)})

# compute statistics
attr_landuse = mask_statistics(land_use, masks, func='mean', weight=pixarea).to_pandas()
attr_landuse.sort_index(axis=1, inplace=True)

# compute main land use
lu_classes = {col: i for i, col in enumerate(attr_landuse.columns, start=1)}
attr_landuse['land_use_main'] = attr_landuse.idxmax(axis=1).map(lu_classes)

# rename attributes
attr_landuse.rename(columns={col: col.split('_')[0] if 'mean' in col else col for col in attr_landuse}, inplace=True)

# plot attributes
plot_attributes(attr_landuse, reservoirs.geometry.x, reservoirs.geometry.y, ncols=4)

### Crop coefficient

In [ ]:
# mapping land use acronym and name
mapping_landuse = {'f': 'forest', 'i': 'irrigated', 'o': 'other'}
variables = ['cropcoef_f', 'cropcoef_i', 'cropcoef_o']

# load maps
crops = xr.Dataset({var: xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')['Band1'].compute() for var in variables})
crops = crops.rename({var: mapping_landuse[var.split('_')[1]] for var in list(crops)})

# mean weighted by the fraction of pixel covered by each land use
crops = crops.to_array('land_use').weighted(land_use.to_array('land_use').fillna(0)).sum('land_use', skipna=True) 
crops = crops.where(~upArea.isnull())
crops.name = 'cropcoef'

# compute statistics
attr_crops = mask_statistics(crops, masks, func=['mean', 'std']).to_pandas()

# plot attributes
plot_attributes(attr_crops, reservoirs.geometry.x, reservoirs.geometry.y, ncols=2)

### Streams

In [ ]:
func = {'chanbnkf': ['mean'],
        'chanbw': ['mean'],
        'changrad': ['mean'],
        'chanlength': ['sum'],
        'chanman': ['mean']}

# load maps
streams = {var: xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')['Band1'].compute() for var in func}
streams = {var: da.rename(var) for var, da in streams.items()}
streams = {var : da.drop([coord for coord in list(da.coords) if coord not in ['lon', 'lat']]) for var, da in streams.items()}
streams = xr.Dataset({var: xr.DataArray(da.data, coords=upArea.coords, name=var) for var, da in streams.items()})

# mask streams (pixels with depth larger than 1 m)
rivers = streams['chanbnkf'] > 1
# rivers.plot(cmap='Blues')
streams = streams.where(rivers)

# calcular estadístico
attr_streams = mask_statistics(streams, masks, func).to_pandas()

# plot attributes
plot_attributes(attr_streams, reservoirs.geometry.x, reservoirs.geometry.y, ncols=5)

### Soil properties

In [ ]:
variables = ['ksat', 'lambda', 'genua', 'soildepth', 'thetas', 'thetar']
layers = [1, 2, 3]
maps = [f'{var}{layer}' for var in variables for layer in layers]

# load maps
soils = {}
for var in tqdm(maps, desc='loading maps'):
    files = list((path_EFAS / 'maps').glob(f'{var}_*.nc'))
    if len(files) > 1:
        ds = {}
        for file in files:
            # type of land use
            cover = mapping_landuse[file.stem.split('_')[1]]
            # import map
            ds[cover] = xr.open_dataset(file)['Band1']#read_static_map(file)
        ds = xr.Dataset(ds)
        da = ds.to_array('land_use').weighted(land_use.to_array('land_use').fillna(0)).sum('land_use', skipna=True)
        soils[var] = da.where(~upArea.isnull())
    elif len(files) == 1:
        soils[var] = xr.open_dataset(files[0])['Band1'] # read_static_map(files[0])
soils = xr.Dataset(soils)

# compute statistics
attr_soils = mask_statistics(soils, masks, func='mean').to_pandas()

# rename attributes
attr_soils.rename(columns={col: col.split('_')[0] for col in attr_soils if 'mean' in col}, inplace=True)

# plot attributes
plot_attributes(attr_soils, reservoirs.geometry.x, reservoirs.geometry.y, ncols=6)

### LAI

I convert the timeseries of 10-daily timesteps into annual averages.

In [ ]:
maps = ['laif', 'laii', 'laio']

# load maps
lai = xr.Dataset({var: xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')['Band1'].compute() for var in maps})
lai = lai.rename({var: mapping_landuse[var[3]] for var in list(lai)})

# mean wheighted by the portion of pixel covered by each land use
lai = lai.to_array('land_use').weighted(land_use.to_array('land_use').fillna(0)).sum('land_use', skipna=True) 
lai = lai.where(~upArea.isnull())
lai.name = 'lai'
lai['time'] = pd.date_range('2021-01-05', periods=len(lai.time), freq='10D')

# monthly resampling
lai_m = lai.resample(time='1M').mean()
lai_m['time'] = [f'{i:02}' for i in range(1, 13)]
lai_agg = xr.Dataset({f'lai{month}': lai_m.sel(time=month).drop('time') for month in lai_m.time.data})

# annual statistics
lai_agg['laiyrmean'] = lai.mean('time')
lai_agg['laiyrmax'] = lai.max('time')
lai_agg['laiyrmin'] = lai.min('time')

# compute statistics
attr_lai = mask_statistics(lai_agg, masks, func=['mean']).to_pandas()

# rename attributes
attr_lai.rename(columns={col: '_'.join(col.split('_')[:-1]) for col in attr_lai if 'mean' in col}, inplace=True)

# plot attributes
plot_attributes(attr_lai, reservoirs.geometry.x, reservoirs.geometry.y, ncols=5)

### Water demand

The original demand maps are monthly time series (domestic, energy, industry, livestock) in mm/day for the period 1990-2023. 

I will compute annual and monthly averages and from those I will compute statistics.

In [ ]:
maps = ['dom', 'ene', 'ind', 'liv']

# load maps
demand = xr.Dataset({var: xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')[var].compute() for var in maps})

# monthly means
demand_m = demand.groupby('time.month').mean('time')

# annual mean
demand_y = demand.groupby('time.year').mean('time').mean('year')
# demand_y = demand.mean('time')

# combine in a single dataset
demand_agg = xr.Dataset()
for key, da in demand_m.items():
    for month in da.month.data:
        demand_agg[f'{key}_{month:02}'] = da.sel(month=month).drop('month')
    demand_agg[f'{key}_yr'] = demand_y[key]

# convert to volume
# demand_agg = demand_agg * 1e-3 * pixarea

# calcular estadístico
attr_demand = mask_statistics(demand_agg, masks, func='sum').to_pandas()

# rename attributes
attr_demand.rename(columns={col: '_'.join(col.split('_')[:-1]) for col in attr_demand if 'sum' in col}, inplace=True)

# plot attributes
plot_attributes(attr_demand[['dom_yr', 'ene_yr', 'ind_yr', 'liv_yr']], reservoirs.geometry.x, reservoirs.geometry.y, ncols=4)

### Reservoirs?

It could be interesting to add two attributes that account for the number of reservoirs upstream and the total storage volume of those reservoirs. I can do it with the reservoirs in EFASv5, but those do not include all the reservoirs in ResOpsES (not to mention all the actual reservoirs).

In [ ]:
# # load map
# var = 'res'
# da = xr.open_mfdataset(f'{path_EFAS}/maps/{var}_*.nc')[var].compute()

# # extract reservoir ID
# ids = np.unique(da)
# ids = ids[~np.isnan(ids)]
# print('EFAS represents {0} reservoirs in the study area'.format(len(ids)))

In [ ]:
# # cargar metadatos de los embalses
# res_efas = pd.read_csv(path_EFAS / 'tables' / 'EFAS_HRES_reservoirs_metadata.csv')
# res_efas.set_index('ResID', inplace=True)

# # recortar a los IDs que hay en el mapa
# res_efas = res_efas.loc[ids, :]

# # convertir en geopandas.GeoDataFrame
# res_efas = gpd.GeoDataFrame(res_efas, geometry=[Point(xy) for xy in zip(res_efas.LisfloodX, res_efas.LisfloodY)], crs=crs)

# # añadir atributos desde las tablas utilizadas LISFLOOD
# for file in glob.glob(f'{path_EFAS}/tables/r*.txt'):
#     var = file.split('\\')[-1].split('.')[0][1:]
#     try:
#         df = pd.read_csv(file, sep=' ', header=None)
#         df.dropna(axis=1, how='all', inplace=True)
#         df.columns = ['ResID', var]
#         df.set_index('ResID', inplace=True, drop=True)
#         res_efas[var] = df
#     except:
#         print(file)
#         continue

# # calcular estadístico
# attr_reservoirs = point_polygon_statistics(res_efas, cuencas, func={'tstor': ['sum', 'count']})
# attr_reservoirs.dropna(axis=1, how='all', inplace=True)
# attr_reservoirs.replace(np.nan, 0, inplace=True)

# attr_reservoirs.columns = ['reserv_vol', 'reserv_count']

# attr_reservoirs.head()

### Lakes?

### Export

In [ ]:
# concatenate all attributes
attrs = pd.concat((attr_geomorphology,
                  attr_landuse,
                  attr_crops,
                  attr_streams,
                  attr_soils,
                  attr_lai,
                  attr_demand,
                  # attr_reservoirs,
                  # attr_lakes,
                 ), axis=1)
attrs.index.name = 'SNCZI'
attrs.sort_index(axis=0, inplace=True)

print('{0} attributes define the characteristics of {1} catchments'.format(*attrs.shape[::-1]))

# export
attrs.to_csv(path_ResOpsES / 'attributes' / 'attributes_EFAS_static_maps.csv')

## LISFLOOD parameters

In this section I load the maps of LISFLOOD calibrated parameters an compute mean catchment values. The results are exported as a CSV file named _attributes_EFAS_model_parameters.csv_.

In [ ]:
# list of calibrated parameters in LISFLOOD
pars = {file.stem: file for file in (path_EFAS / 'parameters').glob('*.nc') if not file.stem.lower().startswith('efas')}
print(f'{len(pars)} parameters were calibrated in EFAS')

# load parameter maps
params = xr.Dataset({par: xr.open_dataset(file)[par].compute() for par, file in pars.items()})

# plot reservoir parameters
reservoir_parameters = ['adjust_Normal_Flood', 'ReservoirRnormqMult']
fig, axes = plt.subplots(ncols=len(reservoir_parameters), figsize=(12, 4), sharex=True, sharey=True)
for ax, par in zip(axes, reservoir_parameters):
    params[par].plot(ax=ax, add_colorbar=True, cmap='magma', cbar_kwargs={'shrink': 0.66})
    ax.scatter(reservoirs.geometry.x, reservoirs.geometry.y, s=2, c='w')
    ax.set_title(par)
    ax.set_aspect('equal')
    ax.axis('off')

# compute statistics
parameters = mask_statistics(params, masks, func='mean').to_pandas()
parameters.index.name = 'SNCZI'
parameters.sort_index(axis=0, inplace=True)
parameters.rename(columns={col: '_'.join(col.split('_')[:-1]) for col in parameters if 'mean' in col}, inplace=True)

# plot parameters
plot_attributes(parameters[reservoir_parameters], reservoirs.geometry.x, reservoirs.geometry.y, ncols=5)
print('{0} LISFLOOD calibrated parameters have been averaged for {1} catchments'.format(*parameters.shape[::-1]))

# export
parameters.to_csv(path_ResOpsES / 'attributes' / 'attributes_EFAS_model_parameters.csv')

## Time series

In this section I used the EMO1 meteorological time series and the EFAS5 discharge reanalysis to create both hydrometeorological attributes (_attributes_EFAS_hydrometeorology.csv_) and the time series that may be used as input in the deep learning model.

### Load time series
#### Meteorology: EMO-1

In [ ]:
# load meteorological time series
path_meteo = path_ResOpsES / 'ancillary' / 'catchstats' / 'meteo'
variables = [x.stem for x in path_meteo.iterdir() if x.is_dir()]
meteo = xr.Dataset({f'{var}_emo1': xr.open_mfdataset(f'{path_meteo}/{var}/*.nc')[f'{var}_mean'].compute() for var in variables})
meteo['time'] = meteo['time'] - np.timedelta64(6, 'h') # WARNING!! One day lag compared with EFAS5
meteo = meteo.rename({'id': 'SNCZI'})

# define attributes
emo1_units = 'e0_emo1: potential evaporation from open water from EMO1 [mm/d]\npr_emo1: observed precipitation from EMO1 [mm/d]\nta_emo1: observed air temperature from EMO1 [°C]\n'
meteo.attrs['Units'] = emo1_units
meteo.time.attrs['timezone'] = 'UTC+00'
meteo.SNCZI.attrs['Description'] = 'The code that identifies a reservoir in the Spanish Inventory of Dams and Reservoirs (Sistema Nacional de Cartografía de Zonas Inundables)'

In [ ]:
# # plot the time series of a specific catchment to check the areal aggregation
# ID = 10
# fig, axes = plt.subplots(nrows=3, figsize=(12, 12), sharex=True)
# for ax, var in zip(axes, ['pr', 'ta', 'e0']):
#     meteo[var].sel(id=ID).plot(ax=ax)

#### EFAS long run

In [ ]:
# load discharge time series
path_discharge = path_ResOpsES / 'ancillary' / 'ncextract' / 'discharge'
dis = xr.open_mfdataset(f'{path_discharge}/*.nc', combine='nested', concat_dim='SNCZI')['dis'].compute()
dis = dis.sel(SNCZI=reservoirs.index)

# Create a CRS variable and set its attributes
crs_attrs = {'epsg_code': 'EPSG:4326',
             'semi_major_axis': 6378137.0,  # WGS 84
             'inverse_flattening': 298.257223563,  # WGS 84
             'grid_mapping_name': 'latitude_longitude'
            }
dis['crs'] = xr.DataArray(data=0, attrs=crs_attrs)  # CRS variable with its attributes

# define attributes
dis.attrs['Units'] = 'dis_efas5: discharge reanalysis (mm/d)\n'
dis.time.attrs['timezone'] = 'UTC+00'
dis.SNCZI.attrs['Description'] = 'The code that identifies a reservoir in the Spanish Inventory of Dams and Reservoirs (Sistema Nacional de Cartografía de Zonas Inundables)'
dis.lat.attrs = {'Units': 'degrees_north',
                 'standard_name': 'latitude',
                 'grid_mapping': 'crs'}
dis.lon.attrs = {'Units': 'degrees_east',
                 'standard_name': 'longitude',
                 'grid_mapping': 'crs'}

# DataArray of catchment area
catch_area = attr_geomorphology['upArea_max'].copy()
catch_area.index.name = 'SNCZI'
catch_area.name = 'CATCH_SM' # in m²
catch_area = xr.DataArray(catch_area)

# compute specific discharge
dis = dis / catch_area * 3600 * 24 * 1000 # mm/d
dis = dis.round(2)
dis.name = 'dis_efas5'

#### Observation

At the model of writing, only the reservoirs from CEDEX are available in the Hydrological Data Management Service (HDMS). The data in HDMS has been quality checked and interpolated to the timezone UTC+00. The data from the Catalan (Agència Catalana del Aigua, ACA) and Andalusian (Hidrosur) agencies are not yet in HDMS, so I will use the raw data.

In [ ]:
start = dis.time[0].data

In [ ]:
# paths to the individual data sets
path_HDMS = path_datasets / 'hDMS' / 'reservoirs'
path_hidrosur = path_datasets / 'Hidrosur' / 'processed' / 'reservoirs'
path_aca = path_datasets / 'ACA' / 'processed' / 'reservoirs'

In [ ]:
# import reservoirs in the Hydrological Data Management Service
hdms = gpd.read_file(path_HDMS / 'reservoirs.shp')
hdms = hdms[['EFAS_ID', 'Name', 'National_S', 'Prov_ID', 'Country-Co', 'Country']]
hdms = hdms.loc[hdms.Prov_ID == '1088']
hdms[['EFAS_ID', 'National_S', 'Prov_ID']] = hdms[['EFAS_ID', 'National_S', 'Prov_ID']].astype(int)
hdms.rename(columns={'National_S': 'LOCAL_ID'}, inplace=True)
hdms.set_index('LOCAL_ID', drop=True, inplace=True)

print('There are {0} reservoirs in HDMS'.format(hdms.shape[0]))

In [ ]:
# obs_variables = ['level', 'volume', 'outflow']

In [ ]:
# load observed time series
obs = {}
for ID in tqdm(reservoirs.index, desc='loading observed timeseries'):
        
    local_ID, provider = reservoirs.loc[ID, ['ID_TS', 'SOURCE_TS']]
    
    if provider == 'CEDEX':
        hdms_ID = hdms.loc[int(local_ID), 'EFAS_ID']
        file = path_HDMS / 'nhoperational24hw' / f'{hdms_ID:04}.nc'
        if file.exists():
            ds = xr.open_dataset(file).sel(ID=hdms_ID)
            ds['ID'] = ID
        else:
            print(f'File {file} does not exist')
    elif provider == 'ACA':
        file = path_aca / 'timeseries' / f'{local_ID}.csv'
        if file.exists():
            df = pd.read_csv(file, usecols=['date', 'level', 'volume'])
            df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
            df['ID'] = ID
            df.set_index(['date', 'ID'], drop=True, inplace=True)
            ds = xr.Dataset.from_dataframe(df).sel(ID=ID)
        else:
            print(f'File {file} does not exist')
    elif provider == 'Hidrosur':
        file = path_hidrosur / 'timeseries' / f'{int(local_ID):03}.csv'
        if file.exists():
            df = pd.read_csv(file)
            df.datetime = pd.to_datetime(df.datetime)
            # select values at the beginning of each day
            df.set_index('datetime', drop=True, inplace=True)
            datemin, datemax = df.index[df.index.hour == 0][[0, -1]]
            idx = pd.date_range(datemin, datemax, freq='D')
            df = df.loc[idx]
            # convert to xarray.Dataset
            df.index.name = 'date'
            df['ID'] = ID
            df.set_index([df.index, df.ID], inplace=True)
            df.drop('ID', axis=1, inplace=True)
            ds = xr.Dataset.from_dataframe(df).sel(ID=ID)
        else:
            print(f'File {file} does not exist')
    else:
        continue
    if ds.date.data[0] < start:
        ds = ds.sel(date=slice(start, None))
    obs[ID] = ds.rename({'ID': 'SNCZI'})

### Attributes

Here I derive the attributes from the time series.

#### EMO-1

In [ ]:
# monthly aggregation (as mean daily value)
meteo_m = meteo.groupby('time.month').mean()
meteo_m = meteo_m.rename({'month': 'time'})

# annual aggregation (as mean daily value)
meteo_y = meteo.mean('time')
meteo_y['time'] = ['year']

# concatenate monthly and annual aggregations
meteo_ag = xr.concat((meteo_m, meteo_y), dim='time')

# organizar en un DataFrame
attr_climate = pd.DataFrame(index=meteo_ag.SNCZI.data)
for var, da in tqdm(meteo_ag.items()):
    # mensual
    df = da.to_pandas().transpose()
    df.columns = [f'{var}_{i:02}' if i != 'year' else f'{var}_{i}' for i in df.columns]
    # concatenar
    attr_climate = pd.concat((attr_climate, df), axis=1)
attr_climate.index.name = 'SNCZI'

In [ ]:
# box plots of monthly and annual meteorological values
ncols = len(meteo)
fig, axes = plt.subplots(ncols=ncols, figsize=(5 * ncols, 3.5))

for ax, var in zip(axes, list(meteo)):
    mask_var = attr_climate.columns.str.startswith(var)
    df = attr_climate.loc[:, mask_var]
    ax.boxplot(df)
    if var in ['pr_emo1', 'e0_emo1']:
        ax.set_ylabel('mm/d')
    elif var in ['ta_emo1']:
        ax.set_ylabel('°C')
    ax.set_title(' '.join(var.split('_')))
    ax.set_xticklabels([i for i in range(1, 13)] + ['yr'])

In [ ]:
# maps of annual meteorological averages
plot_attributes(attr_climate[['pr_emo1_year', 'ta_emo1_year', 'e0_emo1_year']],
                reservoirs.geometry.x,
                reservoirs.geometry.y)

#### EFAS long run

<font color='red'>Add standard deviation or coefficient of variation?</font>

In [ ]:
var = 'dis_efas5'

# mean monthly discharge inflow
dis_m = dis.groupby('time.month').mean('time').to_pandas()
dis_m.columns = [f'{var}_{month:02}' for month in dis_m.columns]

# annual average on minimum, mean and maximum discharge inflow
dis_y = dis.groupby('time.year')
dis_y = pd.DataFrame({f'{var}_yrmean': dis_y.mean().mean('year').to_pandas(),
                      f'{var}_yrmax': dis_y.max().mean('year').to_pandas(),
                      f'{var}_yrmin': dis_y.min().mean('year').to_pandas()})
dis_y.index.name = 'SNCZI'

# combine in a single DataFrame
attr_discharge = pd.concat((dis_m, dis_y), axis=1)

# plot
plot_attributes(attr_discharge[['dis_efas5_yrmean', 'dis_efas5_yrmax', 'dis_efas5_yrmin']], reservoirs.geometry.x, reservoirs.geometry.y, ncols=5)

#### Export

In [ ]:
# concatenate attributes
attr_hydrology = pd.concat((attr_climate, attr_discharge), axis=1)

print('{0} attributes define the hydrometeorology of {1} catchments'.format(*attr_hydrology.shape[::-1]))

# exportar
attr_hydrology.to_csv(path_ResOpsES / 'attributes' / 'atributes_EFAS_hydrometeorology.csv')

### Time series

Here I generate the files in the *timeseries* subfolder of the data set that will be the forcing data in the deep learning model.

<font color='red'>I'm missing the observed data!!!</font>

In [ ]:
# rename variables in 'meteo' and 'dis'
if 'time' in meteo.coords:
    meteo = meteo.rename({'time' : 'date'})
if 'time' in dis.coords:
    dis = dis.rename({'time' : 'date'})

#### CSV

In [ ]:
# output directory
path_ts_csv = path_ResOpsES / 'timeseries' / 'csv' 
path_ts_csv.mkdir(parents=True, exist_ok=True)

for ID in tqdm(reservoirs.index, desc='reservoirs'):

    # extract meteorological time series
    emo = meteo.sel(SNCZI=ID).drop('SNCZI').to_pandas().round(2)

    # extract discharge time series
    efas = dis.sel(SNCZI=ID).drop('SNCZI').to_pandas().round(2)
    efas.name = 'dis_efas5'
    
    # observed time series
    obs_id = obs[ID].drop('SNCZI').to_pandas().round(3)

    # concatenate all time series
    df = pd.concat((obs_id, efas, emo), axis=1)
    df.index.name = 'date'
    
    # export
    df.to_csv(path_ts_csv / f'{ID:04}.csv')

#### NetCDF

In [ ]:
# output directory
path_ts_nc = path_ResOpsES / 'timeseries' / 'netcdf' 
path_ts_nc.mkdir(parents=True, exist_ok=True)

for ID in tqdm(reservoirs.index, desc='reservoirs'):
    
    # concatenar series meteorológicas
    ds = xr.merge((obs[ID].drop('SNCZI'), dis.sel(SNCZI=ID).drop('SNCZI'), meteo.sel(SNCZI=ID).drop('SNCZI')))
    
    # export
    ds.to_netcdf(path_ts_nc / f'{ID:04}.nc')